In [ ]:
from cahos import Scale, VoiceLeading, get_scales, make_scale, make_voice_leading
import polars as pl
import mido
from dataclasses import dataclass
from typing import Tuple

In [ ]:
dis_subseq = [
    [1, 2], [2, 1], [3, 1],
    # [3, 4], [4, 3],
    [6, 1], [1, 6], [5, 1], [1, 5],
    [1, 1], [2, 2],  # [3, 3], [4, 4],
    [5, 5, 5], [6, 6, 6], [7, 7, 7],
]

# 80 is the maximum span we can get with the Cahos ensemble
# Will drop bass by an octave, hence `-12`
base_chords = [s for s in get_scales(
    allowed_intervals=[1, 2, 3, 4, 5, 6, 7],
    disallowed_subsequences=dis_subseq,
    disallowed_beginnings=[],
    max_span=80-12
)]

print(f"number of scales satisfying given constraints: {len(base_chords)}")

In [ ]:
df_base_chords = pl.DataFrame(base_chords)
print(df_base_chords)

In [ ]:
df_selection = df_base_chords.filter(
    pl.col("n_unique_intervals") == 3
).sort(["sequence_entropy", "entropy", "span"], descending=True)
print(df_selection)

In [ ]:
n_shared_notes = 5
upper_limit = 108
lower_limit = 28
min_motion_balance = 3/4
max_n_swaps = 0
max_step_size = 3
n_pseudo_changes = 0

b1 = 40
b2 = 42

voice_leading_opportunities = []
for c1 in df_selection.rows():
    for c2 in df_selection.rows():
        vl = make_voice_leading(c1[0], b1, c2[0], b2)
        if max(vl.midis_a + vl.midis_b) > upper_limit or min(vl.midis_a + vl.midis_b) < lower_limit or vl.motion_balance < min_motion_balance or vl.n_swaps > max_n_swaps:
            continue
        if vl.n_common_notes == n_shared_notes and vl.max_step_size < max_step_size and vl.n_pseudo_changes == n_pseudo_changes:
            voice_leading_opportunities.append(vl)

In [ ]:
df_vl = pl.DataFrame(voice_leading_opportunities)
print(df_vl)

# TODOs

- [ ] create a Phrase or Passage or Line or something class to store a string of chord motions
- [ ] put in instrument ranges and track numbers, program change message to set instruments
- [ ] generate midi messages into appropriate tracks, some tracks may have multiple voices while others have none
- [ ] put in a switch to treat bass as octave lower (midi 35 will be converted to midi 23) can also move all the others up an octave

In [ ]:
@dataclass
class Instrument:
    name: str
    program: int
    register: Tuple[int, int]

piccolo = Instrument("Piccolo", 72, (74, 102))
flute = Instrument("Flute", 73, (60, 96))
oboe = Instrument("Oboe", 68, (58, 91))
clarinet = Instrument("Clarinet", 71, (50, 94))
bass_clarinet = Instrument("Bass Clarinet", 71, (38, 77))
horn = Instrument("Horn", 59, (34, 77))
trombone = Instrument("Trombone", 59, (40, 72))
violin = Instrument("Violin", 44, (55, 103))
viola = Instrument("Viola", 44, (48, 91))
cello = Instrument("Violoncello", 44, (36, 76))
contrabass = Instrument("Contrabass", 44, (28, 67))

ensemble = [piccolo, violin, flute, violin, oboe, viola, clarinet, bass_clarinet, horn, trombone, cello, contrabass]

In [ ]:
mid = mido.MidiFile()
mid.ticks_per_beat = 480
tracks = [mido.MidiTrack() for _ in range(12)]
for t in tracks:
    mid.tracks.append(t)

midis_a = list(df_vl[0]["midis_a"][0][::-1])
midis_b = list(df_vl[0]["midis_b"][0][::-1])
for t, m, i in zip(mid.tracks, midis_a, ensemble):
    while m < i.register[0]:
        m += 12
        print(f"raised octave for {i.name}")
    while m > i.register[1]:
        m -= 12
        print(f"lowered octave for {i.name}")
    t.append(mido.MetaMessage('track_name', name=i.name, time=0))
    t.append(mido.Message('program_change', program=i.program, time=0))
    t.append(mido.Message('note_on', note=m, velocity=64, time=0))
    t.append(mido.Message('note_off', note=m, velocity=127, time=480*4))
for t, m in zip(mid.tracks, midis_b):
    t.append(mido.Message('note_on', note=m, velocity=64, time=0))
    t.append(mido.Message('note_off', note=m, velocity=127, time=480*4))

mid.save('/home/kureta/Downloads/new_song.mid')

In [ ]:
instruments = {}
for t in mid.tracks:
    current_name = ""
    for m in t:
        if m.is_meta and m.type == 'track_name':
            current_name = m.name
        elif m.type == 'program_change':
            instruments[current_name] = m.program

In [ ]:
mid = mido.MidiFile("/home/kureta/Downloads/Untitled score.mid")
mid.print_tracks()